# Purpose


This notebook demonstrates the data pipeline from raw tables to analytical datasets. 



## Imports

In [26]:
from pprint import pprint
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.options.mode.use_inf_as_na = True


In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import warnings
from numba import NumbaDeprecationWarning

warnings.filterwarnings('ignore', message="The default value of regex will change from True to False in a future version.", 
                        category=FutureWarning)

warnings.filterwarnings("ignore", category=NumbaDeprecationWarning)

In [29]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, get_package_path,
    display_as_tabs, string_cleaning, merge_info, initialize_environment,
    list_datasets, load_dataset, save_dataset
)
import great_expectations as ge
import ta_lib.eda.api as eda

In [30]:
initialize_environment(debug=False, hide_warnings=True)

In [31]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
pprint(list_datasets(context))

['/raw/FnB/google_search_data',
 '/raw/FnB/product_manufacturer_list',
 '/raw/FnB/sales_data',
 '/raw/FnB/social_media_data',
 '/raw/FnB/theme_list',
 '/raw/FnB/theme_product_list',
 '/cleaned/FnB/google_search_data',
 '/cleaned/FnB/product_manufacturer_list',
 '/cleaned/FnB/sales_data',
 '/cleaned/FnB/social_media_data',
 '/cleaned/FnB/theme_list',
 '/cleaned/FnB/theme_product_list',
 '/train/FnB/features',
 '/train/FnB/target',
 '/test/FnB/features',
 '/test/FnB/target',
 '/processed/FnB/client_data',
 '/score/FnB/output']


In [32]:
sales_data = load_dataset(context, 'raw/FnB/sales_data')
product_manufacturer_list = load_dataset(context, 'raw/FnB/product_manufacturer_list')
google_search_data = load_dataset(context, 'raw/FnB/google_search_data')
social_media_data = load_dataset(context, 'raw/FnB/social_media_data')
Theme_list = load_dataset(context, 'raw/FnB/theme_list')
Theme_product_list = load_dataset(context, 'raw/FnB/theme_product_list')

### Data Validation

1. Product Id should be an integer
2. Date should be in datetime format in google search data 
3. Date should be in datetime format in social media data 

In [33]:
product_manufacturer_list1 = product_manufacturer_list.copy()
google_search_data1 = google_search_data.copy()
social_media_data1  = social_media_data.copy()

In [34]:
verification_dict = {}
product_manufacturer_list1 = ge.from_pandas(product_manufacturer_list1)
google_search_data1 = ge.from_pandas(google_search_data1)
social_media_data1 = ge.from_pandas(social_media_data1)


In [35]:
verification_dict["rule_1_check"] = product_manufacturer_list1.expect_column_values_to_be_of_type("PRODUCT_ID", "int64", mostly=None, 
                                             result_format="BASIC", include_config=True).to_json_dict()

if verification_dict["rule_1_check"]["success"]:
    print("Rule 1 passed")
else:
    print("Rule 1 failed")

Rule 1 passed


In [36]:

verification_dict["rule_2_check"] = google_search_data1.expect_column_values_to_be_of_type("date", "datetime64", mostly=None, 
                                             result_format="BASIC", include_config=True).to_json_dict()

if verification_dict["rule_2_check"]["success"]:
    print("Rule 2 passed")
else:
    print("Rule 2 failed")

Rule 2 failed


In [37]:
verification_dict["rule_3_check"] = social_media_data1.expect_column_values_to_be_of_type("published_date", "datetime64", mostly=None, 
                                             result_format="BASIC", include_config=True).to_json_dict()

if verification_dict["rule_3_check"]["success"]:
    print("Rule 3 passed")
else:
    print("Rule 3 failed")

Rule 3 failed


## Cleaning

### Sales table

From data discovery, we know the following

* No column are duplicates
* key columns: PRODUCT_ID
* integer columns: product_id , sales_units_value , sales_lbs_value , system_calendar_key_N)
* float columns: sales_dollars_value 
* object columns:  Vendors
* date columns:system_calendar_key_n

In [38]:
sales_data_clean = (
    sales_data
    .copy()
    .change_type(['sales_dollars_value'], np.int64)
    .to_datetime('system_calendar_key_N', format='%Y%m%d')
    .rename_columns({'system_calendar_key_N': 'date'})                                                                                                                       
    .clean_names(case_type='snake')
)
sales_data_clean.head()

,date,product_id,sales_dollars_value,sales_units_value,sales_lbs_value
0,2016-01-09,1,13927,934,18680
1,2016-01-09,3,10289,1592,28646
2,2016-01-09,4,357,22,440
3,2016-01-09,6,23113,2027,81088
4,2016-01-09,7,23177,3231,58164


### Social Media

From data discovery, we know the following

* No column are duplicates
* 41%age values are missing in ThemeId column
* key columns: Theme Id
* integer columns: total_post
* float columns: Theme Id
* object columns:  published_date
* date columns: published_date

In [39]:
social_media_data_clean = (
    social_media_data
    .copy()
    .replace({'': np.NaN})
    .dropna(axis = 0)
    .change_type(['Theme Id'], np.int64)
    .to_datetime('published_date')
    .rename_columns({'Theme Id': 'claim_id' ,'published_date' : 'date' } )                                                                                                                               
    .clean_names(case_type='snake')
    .drop_duplicates()
)

social_media_data_clean.head(5)

,claim_id,date,total_post
0,148,2015-01-10,76
1,148,2015-10-10,31
2,148,2015-11-10,65
3,148,2015-12-10,88
4,148,2015-10-13,85


### Theme Product List

From data discovery, we know the following

* No column are duplicates
* key columns: CLAIM_ID
* integer columns: CLAIM_ID , PRODUCT_ID
* This will go into production code

In [40]:
theme_product_list_clean= (
    Theme_product_list
    .copy()                                                                                                                               
    .clean_names(case_type='snake')
)
theme_product_list_clean.head()

,product_id,claim_id
0,26,8
1,29,8
2,48,81
3,50,81
4,74,227


### Product Manufacturer List

From data discovery, we know the following

* key columns: PRODUCT_ID
* integer columns: PRODUCT_ID 
* object columns:  Vendors
* This will go into production code

In [41]:
product_manufacturer_list_clean = (
    product_manufacturer_list
    .copy()
    .drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], axis=1)                                                                                                               
    .clean_names(case_type='snake')
)
product_manufacturer_list_clean.head(5)

,product_id,vendor
0,1,Others
1,2,Others
2,3,Others
3,4,Others
4,5,Others


### Themes List

From data discovery, we know the following

* No column are duplicates
* key columns: CLAIM_ID
* integer columns: CLAIM_ID
* object columns:  Claim Name
* This will go into production code

In [42]:
theme_list_clean = (
    Theme_list
    .copy()
    .passthrough()    
    .replace({'': np.NaN})
    .clean_names(case_type='snake')
)
theme_list_clean.head(5)

,claim_id,claim_name
0,0,No Claim
1,8,low carb
2,15,beans
3,16,cocoa
4,26,vanilla


### Google search data

From data discovery, we know the following

* date column has type date and others columns has dtype int  : nothing to fix. Apply generic cleaning (strip extra whitespace etc)
* No column are duplicates
* Claim_id is key column : ensure no duplicate values
* This will go into production code

In [43]:
google_search_data_clean = (
    google_search_data
    .copy()
    .replace({'': np.NaN})
    .to_datetime('date', format='%d-%m-%Y')
    .sort_values(by=['searchVolume'],ascending=False)
    .rename_columns({'year_new' : 'year'})
    .drop_duplicates(subset = ['date' , 'Claim_ID' , 'platform'],keep = 'first').reset_index(drop = True)
    .clean_names(case_type='snake')    
)
google_search_data_clean.head(5)

,date,platform,search_volume,claim_id,week_number,year
0,2018-01-10,google,370484,657,2,2018
1,2018-01-09,google,259069,657,2,2018
2,2018-01-11,google,197772,657,2,2018
3,2016-10-30,google,169543,158,43,2016
4,2015-10-02,google,145848,158,40,2015


In [44]:
save_dataset(context, sales_data_clean, "cleaned/FnB/sales_data")
save_dataset(context, social_media_data_clean, "cleaned/FnB/social_media_data")
save_dataset(context, google_search_data_clean, "cleaned/FnB/google_search_data")
save_dataset(context, theme_product_list_clean, "cleaned/FnB/theme_product_list")
save_dataset(context, theme_list_clean, "cleaned/FnB/theme_list")
save_dataset(context, product_manufacturer_list_clean, "cleaned/FnB/product_manufacturer_list")